In [1]:
import pandas as pd

# Load the Excel file
df = pd.read_csv(r'C:\Users\KamranImtiyaz\OneDrive - MinoriLabs\Desktop\Agent\New Ecom - Industry Segmentation _ Oct2025(WIP).csv')

# Assuming the column with websites is named 'Website'
websites = df['Website'].tolist()

In [2]:
len(websites)

32043

In [ ]:
websites[:20]

In [ ]:
from langchain_perplexity import ChatPerplexity
from creds import perplexity_api_key

# Initialize with your API key (it will pick from env variable if set)
chat_perplexity = ChatPerplexity(model="sonar-medium", temperature=0.2, api_key=perplexity_api_key, max_tokens=1000)


In [ ]:
def classify_with_perplexity(url):
    prompt = (
    f"Analyze the website at {url}. "
    "Evaluate its structure, content, and functionality to determine its primary purpose. "
    "Classify the website into one of the following categories: "
    "1) 'e-commerce' — if it primarily enables online sales or transactions and includes features such as product listings, shopping carts, checkout or payment gateways, customer accounts, promotional offers, or order tracking. "
    "2) 'mark-ops' — if it focuses on marketing, branding, lead generation, or corporate communications, typically featuring sections like 'About Us', 'Services', 'Case Studies', 'Blog', or 'Contact Us' but without direct purchase options. "
    "3) 'other' — if it does not clearly fit into either category. "
    "Respond with only one of the following labels: 'e-commerce', 'mark-ops', or 'other'."
    )

    response = chat_perplexity.invoke(
        input=prompt,

        )
    return response.content.strip().lower()

In [5]:
from google.genai import types
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch
import json
google_search_tool = Tool(google_search=GoogleSearch())

from goog import GOOGLE_API_KEY

client = genai.Client(api_key=GOOGLE_API_KEY)

def classify_with_gemini(url):
    prompt = (
        f"Analyze the website at {url}. "
        "Evaluate its structure, content, and functionality to determine its primary purpose. "
        "Classify the website into one of the following categories: "
        "1) 'e-commerce' — if it primarily enables online sales or transactions and includes features such as product listings, shopping carts, checkout or payment gateways, customer accounts, promotional offers, or order tracking. "
        "2) 'mark-ops' — if it focuses on marketing, branding, lead generation, or corporate communications, typically featuring sections like 'About Us', 'Services', 'Case Studies', 'Blog', or 'Contact Us' but without direct purchase options. "
        "3) 'other' — if it does not clearly fit into either category or its a nan"
        "Respond with only one of the following labels: 'e-commerce', 'mark-ops', or 'other'."
    )
    thinking_budget = 5000  # @param {type:"slider", min:0, max:24576, step:1}

    response = client.models.generate_content(
        model="gemini-2.5-flash-preview-09-2025",
        contents=prompt,
        config=GenerateContentConfig(
            tools=[google_search_tool],
            thinking_config=types.ThinkingConfig(
                thinking_budget=thinking_budget,
                include_thoughts=True
            ),
        )
    )
    print(response)

    # Updated error handling
    if response is None or getattr(response, "text", None) is None:
        return ""
    return response.text.strip().lower()

In [ ]:
# ... existing code ...
df.loc[:5, "Category_perplexity"] = df.loc[:5, "Website"].apply(classify_with_perplexity) 
# ... existing code ...

In [6]:
df.loc[:15, "Category_Gemini"] = df.loc[:15, "Website"].apply(classify_with_gemini)

sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        text="""**Analyzing Website Focus**

I'm currently focused on dissecting the website's structure and content to pinpoint its core function. I'm actively analyzing search results for transaction-related keywords, contrasting those with informational or promotional terms. This should allow me to categorize the website as 'e-commerce', 'mark-ops', or something else entirely.


""",
        thought=True
      ),
      Part(
        text="""**Attempting Content Discovery**

I'm hitting a roadblock. My initial probes for the website's inner workings are failing. No structural or content data is surfacing. I'll switch gears and try a more targeted approach, seeking out the company or website's own descriptions.


""",
        thought=True
      ),
      Part(
        text="""**Examining Website Availability**

I'm frustrated. The second set of search queries were

In [7]:
df.head(20)

,Company Name,Website,Company Linkedin Url,Annual Revenue,Industry,Segmented Industry,Product/ Service/ Offering,Company Country,Remarks,Category_Gemini
0,!n-joy Carllife,http://www.injoycarlife.com,http://www.linkedin.com/company/-n-joy-carllife,256200000,automotive,NaN,NaN,United States,NaN,other
1,!Viva la Mama!,http://www.vivalamama.org,http://www.linkedin.com/company/-viva-la-mama-,174500000,"health, wellness & fitness",NaN,NaN,United States,NaN,"the website's structure, which includes sectio..."
2,"""Apollo""​ Flow Controls, Manufactured by Conbr...",http://www.aalberts-ips.us,http://www.linkedin.com/company/conbraco-indus...,NaN,NaN,NaN,NaN,United States,NaN,the website exhibits the following characteris...
3,#CaliforniansForAll College Corps,http://www.cacollegecorps.com,http://www.linkedin.com/company/californiansfo...,146000000,nonprofit organization management,NaN,NaN,United States,NaN,mark-ops
4,(ISSA) International Sports Sciences Association,http://www.issaonline.com,http://www.linkedin.com/company/issaonline,100000000,government administration,NaN,NaN,United States,NaN,e-commerce
5,(RED),http://www.red.org,http://www.linkedin.com/company/-red-,800000000,nonprofit organization management,NaN,NaN,United States,NaN,mark-ops
6,(WIS) Wellbore Integrity Solutions,http://www.wellboreintegrity.com,http://www.linkedin.com/company/wellbore-integ...,319300000,oil & energy,NaN,NaN,United States,NaN,mark-ops
7,.,http://www.asconoil.com,http://www.linkedin.com/company/oil-company-ltd,678300000,oil & energy,NaN,NaN,United States,NaN,mark-ops
8,.Com Marketing,http://www.commarketing.com,http://www.linkedin.com/company/.com-marketing,128400000,marketing & advertising,NaN,NaN,United States,NaN,the website at http://www.commarketing.com (wh...
9,.NET Foundation,http://www.dotnetfoundation.org,http://www.linkedin.com/company/dotnetfoundation,1000000000,information technology & services,NaN,NaN,United States,NaN,other


In [18]:
!"C:\Program Files\Python312\python.exe" -m pip install compressed-tensors --upgrade --force-reinstall


Defaulting to user installation because normal site-packages is not writeable
  Using cached compressed_tensors-0.12.2-py3-none-any.whl.metadata (7.0 kB)
  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached pydantic-2.12.4-py3-none-any.whl.metadata (89 kB)
  Using cached loguru-0.7.3-py3-none-any.whl.metadata (22 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached mpmath-1.3.0-py3-none-any.w

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-core 0.3.28 requires packaging<25,>=23.2, but you have packaging 25.0 which is incompatible.

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from hf import hf_api_key
import os 
os.environ["HF_API_KEY"] = hf_api_key

In [13]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer

import torch
import re

# Load model and tokenizer
model_name = "moonshotai/Kimi-K2-Thinking"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, dtype="auto",device_map="auto")

# Define classification function
def classify_with_kimi(url):
    prompt = (
        f"Analyze the website at {url}. "
        "Evaluate its structure, content, and functionality to determine its primary purpose. "
        "Classify the website into one of the following categories: "
        "1) 'e-commerce' — if it primarily enables online sales or transactions and includes features such as product listings, shopping carts, checkout or payment gateways, customer accounts, promotional offers, or order tracking. "
        "2) 'mark-ops' — if it focuses on marketing, branding, lead generation, or corporate communications, typically featuring sections like 'About Us', 'Services', 'Case Studies', 'Blog', or 'Contact Us' but without direct purchase options. "
        "3) 'other' — if it does not clearly fit into either category. "
        "Respond with only one of the following labels: 'e-commerce', 'mark-ops', or 'other'."
    )

    try:
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=100,
                temperature=0.3,
                do_sample=True,
            )
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        match = re.search(r"(e-commerce|mark-ops|other)", result.lower())
        return match.group(1) if match else "other"
    except Exception as e:
        print(f"Error processing {url}: {e}")
        return "error"

# Example: Load your DataFrame
# df = pd.read_csv("websites.csv")

# Run classification on first 15 rows
df.loc[:15, "Category_Kimi"] = df.loc[:15, "Website"].apply(classify_with_kimi)

# Save results if you want
# df.to_csv("classified_websites.csv", index=False)


You are using a model of type kimi_k2 to instantiate a model of type deepseek_v3. This is not supported for all configurations of models and can yield errors.


ImportError: compressed_tensors is not installed and is required for compressed-tensors quantization. Please install it with `pip install compressed-tensors`.

In [1]:
import os
import time
import threading
import pandas as pd
import requests
import json
import sys
from Open import OPEN_AI_API_KEY

# pip install openai requests pandas
import openai

# ========================================================
# 0. Configuration (edit as needed)
# ========================================================
OPENAI_API_KEY = os.environ.get("OPEN_AI_API_KEY")
if not OPENAI_API_KEY:
    print("ERROR: Set the OPENAI_API_KEY environment variable and re-run.")
    sys.exit(1)

openai.api_key = OPENAI_API_KEY
MODEL = "gpt-4o-mini"   # function-calling capable model in this flow
CSV_PATH = r"C:\Users\KamranImtiyaz\OneDrive - MinoriLabs\Desktop\Agent\New Ecom - Industry Segmentation _ Oct2025(WIP).csv"
OUTPUT_PATH = r"classified_output.csv"

# ========================================================
# 0b. Timeout wrapper (Windows-compatible)
# ========================================================
def run_with_timeout(func, args=(), kwargs=None, timeout=10):
    if kwargs is None:
        kwargs = {}
    result = [None]
    exception = [None]

    def target():
        try:
            result[0] = func(*args, **kwargs)
        except Exception as e:
            exception[0] = e

    thread = threading.Thread(target=target, daemon=True)
    thread.start()
    thread.join(timeout)

    if thread.is_alive():
        return "TIMEOUT"
    if exception[0]:
        raise exception[0]
    return result[0]

# ========================================================
# 1. Load CSV (preserve previous run if exists)
# ========================================================
if os.path.exists(OUTPUT_PATH):
    print(f"Found existing output file -> loading from {OUTPUT_PATH} to preserve previous labels.")
    df = pd.read_csv(OUTPUT_PATH)
else:
    df = pd.read_csv(CSV_PATH)

if "Category_Gemini" not in df.columns:
    df["Category_Gemini"] = ""

# normalize column values to be robust for skip logic
df["Category_Gemini"] = df["Category_Gemini"].astype(str).str.strip()
df["Category_Gemini"].replace({"nan": "", "NaN": "", "None": "", "none": ""}, inplace=True)

# ========================================================
# 2. Manual Start Options (edit MANUAL_START / MANUAL_ROW as needed)
# ========================================================
MANUAL_START = True
MANUAL_ROW = 286

if MANUAL_START:
    start_index = MANUAL_ROW
    print(f"Manual start enabled → Starting at row {start_index}")
else:
    empty_mask = df["Category_Gemini"].isna() | (df["Category_Gemini"].astype(str).str.strip() == "")
    if empty_mask.any():
        # idxmax returns first True index because booleans cast to ints; safe here
        start_index = empty_mask.idxmax()
    else:
        start_index = None
    print(f"Auto-resume enabled → Starting at first unclassified row: {start_index}")

if start_index is None or pd.isna(start_index):
    print("✔ All rows already classified!")
    sys.exit(0)

# ========================================================
# 3. Simple "tool" implementations
#    - perform_search(q): returns short aggregated snippets from Google search results
#      (lightweight, scraping Google SERP HTML; not a heavy production search integration)
# ========================================================
HEADERS = {"User-Agent": "Mozilla/5.0 (compatible; classification-bot/1.0)"}

def perform_search(query, num_results=5):
    """
    Lightweight scraping of Google results page. Returns a concatenated string of titles+snippets.
    NOTE: This is a basic approach for small scale use. For production use a proper search API (SerpAPI, Bing, GCS).
    """
    try:
        if not query or str(query).strip() == "":
            return "NO_QUERY"
        # Use simple Google query URL (may be rate-limited or blocked). Keep it conservative.
        url = "https://www.google.com/search"
        params = {"q": query, "num": num_results, "hl": "en"}
        r = requests.get(url, params=params, headers=HEADERS, timeout=10)
        if r.status_code != 200:
            return f"ERROR_HTTP_{r.status_code}"
        html = r.text
        # Very small, robust parsing: get lines that contain "class=\"BNeawe vvjwJb AP7Wnd\"" (titles) or "class=\"BNeawe s3v9rd AP7Wnd\"" (snippets)
        # Google markup changes frequently; fallback to returning raw HTML trimmed.
        snippets = []
        # naive split searching for typical snippet markers:
        for token in ['<div class="BNeawe vvjwJb AP7Wnd">', '<div class="BNeawe s3v9rd AP7Wnd">', '<div class="BNeawe s3v9rd AP7Wnd">']:
            if token in html:
                parts = html.split(token)[1: num_results*2 + 1]
                for part in parts:
                    text = part.split("</div>", 1)[0]
                    # remove tags roughly
                    txt = ''.join(c for c in text if c not in '<>/')
                    snippets.append(txt.strip())
        if not snippets:
            # fallback: return the first 3000 characters of page as "search result"
            return html[:3000]
        return "\n".join(snippets)[:3000]
    except Exception as e:
        return f"ERROR_SEARCHING: {e}"

def fetch_website_html(url):
    """Robust website fetcher returning HTML or an error code string"""
    if pd.isna(url) or str(url).strip() == "":
        return "NO_URL"
    try:
        r = requests.get(url, headers=HEADERS, timeout=12)
        if r.status_code != 200:
            return f"ERROR_HTTP_{r.status_code}"
        text = r.text
        if not text or len(text.strip()) == 0:
            return "NO_HTML"
        return text
    except Exception as e:
        return f"ERROR_FETCHING_HTML: {e}"

# ========================================================
# 4. OpenAI function-calling helper
#    We'll expose one function "perform_search" to the model. If model calls it, we run it and re-invoke.
# ========================================================
FUNCTIONS = [
    {
        "name": "perform_search",
        "description": "Perform a web search and return short aggregated titles/snippets. Input is a simple query string.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "Search query"},
                "num_results": {"type": "integer", "description": "Number of results to return", "default": 5}
            },
            "required": ["query"]
        }
    }
]

SYSTEM_INSTRUCTIONS = (
    "You are a classifier assistant. For each website you must respond with exactly one token: "
    "'e-commerce', 'mark-ops', or 'other'.\n\n"
    "You may call the tool 'perform_search' to fetch web search snippets that will help classification. "
    "If you call the tool, you will receive the tool output and should then return only the final label string."
)

def classify_with_openai(url):
    """
    Use OpenAI Chat Completion with function-calling flow (single tool).
    Returns one of: 'e-commerce', 'mark-ops', 'other'
    """
    if pd.isna(url) or str(url).strip() == "":
        return "other"

    # fetch partial website HTML for context (trimmed)
    html = fetch_website_html(url)
    html_trimmed = html[:5000] if isinstance(html, str) else str(html)[:5000]

    user_prompt = (
        f"Analyze this website URL: {url}\n\n"
        "I will provide the site's HTML (trimmed) below. Use the HTML and, if needed, call the perform_search tool to fetch web search snippets.\n\n"
        "Website HTML (trimmed):\n"
        "---------------- HTML START ----------------\n"
        f"{html_trimmed}\n"
        "---------------- HTML END ----------------\n\n"
        "Classify into one of: 'e-commerce', 'mark-ops', 'other'. Answer with only the one-word label and nothing else."
    )

    try:
        # initial call; allow the model to call the tool
        resp = openai.ChatCompletion.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": SYSTEM_INSTRUCTIONS},
                {"role": "user", "content": user_prompt},
            ],
            functions=FUNCTIONS,
            function_call="auto",
            max_tokens=400,
            temperature=0.0,
        )
    except Exception as e:
        # If API request fails, fallback to "other"
        print(f"OpenAI API error: {e}")
        return "other"

    # parse response
    message = resp["choices"][0]["message"]

    # if model asked to call a function:
    if message.get("function_call"):
        fname = message["function_call"]["name"]
        fargs_json = message["function_call"].get("arguments", "{}")
        try:
            fargs = json.loads(fargs_json)
        except Exception:
            fargs = {}
        # Execute only perform_search for now
        if fname == "perform_search":
            q = fargs.get("query") or url
            num = int(fargs.get("num_results", 5) or 5)
            tool_out = perform_search(q, num_results=num)

            # send the function result back to the model
            try:
                followup = openai.ChatCompletion.create(
                    model=MODEL,
                    messages=[
                        {"role": "system", "content": SYSTEM_INSTRUCTIONS},
                        {"role": "user", "content": user_prompt},
                        {
                            "role": "assistant",
                            "content": None,
                            "function_call": {"name": "perform_search", "arguments": json.dumps(fargs)}
                        },
                        {"role": "function", "name": "perform_search", "content": tool_out}
                    ],
                    max_tokens=200,
                    temperature=0.0
                )
            except Exception as e:
                print(f"OpenAI followup error: {e}")
                return "other"

            final_msg = followup["choices"][0]["message"]
            final_text = (final_msg.get("content") or "").strip().lower()
            # ensure it's one of expected labels
            if final_text in ["e-commerce", "mark-ops", "other"]:
                return final_text
            # sometimes model returns JSON or extra whitespace/newlines
            # find if any expected label appears inside the output
            for label in ["e-commerce", "mark-ops", "other"]:
                if label in final_text:
                    return label
            # fallback
            return "other"

    else:
        # model responded directly (no tool call)
        text = (message.get("content") or "").strip().lower()
        if text in ["e-commerce", "mark-ops", "other"]:
            return text
        for label in ["e-commerce", "mark-ops", "other"]:
            if label in text:
                return label
        return "other"

# ========================================================
# 5. Batch processing / main loop
# ========================================================
BATCH_SIZE = 15
TOTAL = len(df)

print(f"Total rows: {TOTAL}")
print(f"Processing from row {start_index}...\n")

for batch_start in range(start_index, TOTAL, BATCH_SIZE):
    batch_end = min(batch_start + BATCH_SIZE, TOTAL)

    # clear screen on each batch (Windows 'cls', else 'clear')
    os.system('cls' if os.name == 'nt' else 'clear')

    print(f"\n==============================")
    print(f"PROCESSING BATCH {batch_start} → {batch_end - 1}")
    print(f"==============================\n")

    for i in range(batch_start, batch_end):
        old_value = str(df.at[i, "Category_Gemini"]).strip().lower()

        # final skip logic - skip if already labeled (non-empty)
        if old_value not in ["", "nan", "none"]:
            print(f"⏩ Row {i} already classified ({old_value}) → Skipping")
            continue

        url = df.at[i, "Website"]
        print(f"\n🔵 Row {i} — URL: {url}")

        try:
            category = run_with_timeout(classify_with_openai, args=(url,), timeout=30)

            if category == "TIMEOUT":
                print("⏱️ Timeout reached — skipping this URL")
                category = "timeout"

        except Exception as e:
            print(f"❌ Error while classifying row {i}: {e}")
            category = "other"

        df.at[i, "Category_Gemini"] = category
        print(f"✅ Classified as: {category}\n")

    # Save progress to OUTPUT_PATH so future runs preserve these values
    df.to_csv(OUTPUT_PATH, index=False)
    print(f"💾 Progress saved to → {OUTPUT_PATH}")

    print("\n⏳ Sleeping for 30 seconds...\n")
    time.sleep(30)

print("\n🎉 ALL DONE — Entire CSV classified successfully!")


ERROR: Set the OPENAI_API_KEY environment variable and re-run.


SystemExit: 1

C:\Users\KamranImtiyaz\AppData\Roaming\Python\Python314\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
